In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Tải dữ liệu từ Drive vào Google Colab

In [0]:
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m_a.npy .
!cp /content/drive/My\ Drive/DuLieu/filter_feature/m_a_l.npy .

In [3]:
from scipy.signal import butter, lfilter, filtfilt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import keras
from keras.layers import Dropout
from keras.layers import Activation
from shutil import copyfile
import gc
import os
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal
kf = KFold(n_splits=5, shuffle= True, random_state=28)
from scipy import stats
from sklearn.model_selection import train_test_split
import librosa
import IPython.display as ipd

Using TensorFlow backend.


In [0]:
def create_model():
  model = Sequential()
  model.add(LSTM(64, return_sequences=True))
  model.add(LSTM(64))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(loss='binary_crossentropy', optimizer= keras.optimizers.Adam(lr = 0.0005), metrics=['accuracy'])
  return model

In [0]:
class LogConfusionMatrix(keras.callbacks.Callback):
  def __init__(self, names):
    self.names = names
  def on_train_begin(self, logs={}):
    self.bestacc = 0.0
    self.bestcf = None
  def on_epoch_end(self, epoch, logs = {}):
    if logs["val_acc"] > self.bestacc:
      self.bestacc = logs["val_acc"] 
      y_pred = self.model.predict(self.validation_data[0])
      y_pred = [1 * (x[0]>=0.5) for x in y_pred]
      self.bestcf = confusion_matrix(self.validation_data[1], y_pred)
  def on_train_end(self, logs = {}):
    f = open(self.names, "a")
    f.write("Best val_acc: " + str(self.bestacc))
    f.write("\n")
    f.write("Best cf: \n" + str(self.bestcf))
    f.write("\n")
    f.close()


In [0]:
def train_case_2(X_aug, y_aug, filename, epochs = 40):
  i = 1
  for train_index, test_index in kf.split(X_aug):
    print("Cross Valid: " + str(i))
    f = open(filename, "a")
    f.write("Cross Valid: " + str(i))
    f.write("\n")
    f.close()
    logcf = LogConfusionMatrix(filename)
    es = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.008, patience=9, verbose=1, mode='auto')
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = "loss",patience = 3, factor = 0.8, min_lr = 0.0001, verbose = 1)
    model = create_model()
    mean = np.mean(X_aug[train_index], axis = 0)
    std = np.std(X_aug[train_index], axis = 0)
    X_train = (X_aug[train_index] - mean) / std
    y_train = y_aug[train_index]
    X_test = (X_aug[test_index] - mean) / std
    y_test = y_aug[test_index]
    model.fit(X_train, y_train, epochs = epochs, batch_size = 64, verbose = 1, validation_data = (X_test, y_test), callbacks = [es, logcf, reduce_lr])
    #print(confusion_matrix(y_test, y_pred))
    gc.collect()
    i += 1

In [0]:
X = np.load("m_a.npy")
y = np.load("m_a_l.npy")

In [8]:
train_case_2(X, y, "mfcc_case_2.txt", 40)

Cross Valid: 1





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 30968 samples, validate on 7742 samples
Epoch 1/40





30968/30968 [==============================] - 110s 4ms/step - loss: 0.3316 - acc: 0.8586 - val_loss: 0.2202 - val_acc: 0.9169
Epoch 2/40
30968/30968 [==============================] - 106s 3ms/step - loss: 0.1647 - acc: 0.9402 - val_loss: 0.1372 - val_acc: 0.9490
Epoch 3/40
30968/30968 [==============================] - 105s 3ms/step - loss: 0.1068 - acc: 0.9633 - val_loss: 0.1026 - val_acc: 0.9656
Epoch 4/40
30968/30968 [==============================] - 105s 3ms/step - loss: 0.0730 - acc: 0.9762 - val_loss: 0.0754 - val_acc: 0.9764
Epoch 5/40
30968/30968 [==============================] - 105s 3ms/step - loss: 0.0523 - acc: 0.9833 - val_loss: 0.0658 - val_acc: 0.9784
Epoch 6/40
30968/30968 [==============================] - 105s 3ms/step - loss: 0.0397 - acc: 0.9867 - val_loss: 0.0662 - val_acc: 0.9762
E